In [20]:
import pandas as pd
import ccxt
import os
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST 
from alpaca_trade_api.rest import TimeFrame
from alpaca_trade_api.rest import TimeFrameUnit
import random
import krakenex

In [6]:
load_dotenv()

ALPACA_API_KEY_ID = os.getenv('ALPACA_API_KEY_ID')
ALPACA_API_SECRET_KEY = os.getenv('ALPACA_API_SECRET_KEY')

KRAKEN_API_KEY_ID =  os.getenv('KRAKEN_API_KEY_ID')
KRAKEN_API_SECRET_KEY = os.getenv('KRAKEN_API_SECRET_KEY')

In [37]:
#Kraken

kraken_api = krakenex.API(KRAKEN_API_KEY_ID, KRAKEN_API_SECRET_KEY)

#print(kraken_api.query_public("AssetPairs")["result"])

kraken = ccxt.kraken({
    'apiKey': KRAKEN_API_KEY_ID,
    'secret': KRAKEN_API_SECRET_KEY,
})

markets = kraken.load_markets()

print(kraken)
if (kraken.has['fetchTickers']):
    print(kraken.fetch_tickers(['BTC/USD','ETH/USD','LUNA/USD','SOL/USD','XRP/USD','ADA/USD',
    'AVAX/USD','DOT/USD','DOGE/USD','SHIB/USD','MATIC/USD',
    'DAI/USD','LTC/USD','ATOM/USD','LINK/USD','UNI/USD','TRX/USD','BCH/USD'])) 

Kraken
{'ADA/USD': {'symbol': 'ADA/USD', 'timestamp': 1649291328791, 'datetime': '2022-04-07T00:28:48.791Z', 'high': 1.158978, 'low': 1.049094, 'bid': 1.064247, 'bidVolume': None, 'ask': 1.064589, 'askVolume': None, 'vwap': 1.113301, 'open': 1.053411, 'close': 1.064072, 'last': 1.064072, 'previousClose': None, 'change': 0.010661, 'percentage': 1.0120456308126646, 'average': 1.0587415, 'baseVolume': 17284680.25282928, 'quoteVolume': 19243051.81015509, 'info': {'a': ['1.064589', '1037', '1037.000'], 'b': ['1.064247', '3132', '3132.000'], 'c': ['1.064072', '699.65937000'], 'v': ['175409.33716641', '17284680.25282928'], 'p': ['1.057974', '1.113301'], 't': ['354', '13011'], 'l': ['1.051236', '1.049094'], 'h': ['1.068369', '1.158978'], 'o': '1.053411'}}, 'ATOM/USD': {'symbol': 'ATOM/USD', 'timestamp': 1649291328791, 'datetime': '2022-04-07T00:28:48.791Z', 'high': 29.3584, 'low': 26.7205, 'bid': 27.093, 'bidVolume': None, 'ask': 27.108, 'askVolume': None, 'vwap': 28.037498, 'open': 26.8154, '

In [10]:
#Alpaca 

alpaca_api = REST()

#Facebook, Amazon, Apple, Netflix, Google, Microsoft, Uber, Lyft, Airbnb

FANNGMULA_tickers = ["META", "AMZN", "AAPL", "NTFX", "GOOG", "MSFT", "UBER", "LYFT", "ABNB"]

alpaca_api.get_bars(FANNGMULA_tickers, TimeFrame(45, TimeFrameUnit.Minute), "2021-06-08", "2021-06-08", adjustment='raw').df

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-06-08 07:30:00+00:00,126.10,126.16,125.9600,126.06,20951,304,126.049447,AAPL
2021-06-08 08:15:00+00:00,126.05,126.30,126.0500,126.30,21181,349,126.231904,AAPL
2021-06-08 09:00:00+00:00,126.27,126.32,126.2200,126.28,15955,308,126.284120,AAPL
2021-06-08 09:45:00+00:00,126.29,126.40,125.9000,125.90,30179,582,126.196877,AAPL
2021-06-08 10:30:00+00:00,125.90,126.75,125.8400,126.75,105380,1376,126.530863,AAPL
...,...,...,...,...,...,...,...,...
2021-06-08 20:15:00+00:00,49.81,49.88,49.7800,49.86,21235,97,49.860162,UBER
2021-06-08 21:00:00+00:00,49.86,49.88,49.8600,49.86,44362,30,49.879444,UBER
2021-06-08 21:45:00+00:00,49.86,49.88,49.8000,49.85,1739,12,49.864410,UBER
